# Example: Compare ASCAT SM against ISMN

In [ ]:
# install the ISMN package first https://github.com/TUW-GEO/ismn
import ismn.interface as ismn 
import warnings

# install the ascat package first https://github.com/TUW-GEO/ascat
from ascat.read_native.cdr import AscatGriddedNcTs

import pytesmo.temporal_matching as temp_match
import pytesmo.scaling as scaling
import pytesmo.df_metrics as df_metrics
import pytesmo.metrics as metrics

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

Create the ascat reader:

In [ ]:
testdata_path = Path(".").resolve().parent.parent / "tests" / "test-data"
ascat_data_folder = testdata_path / "sat" / "ascat" / "netcdf" / "55R22"
ascat_grid_fname = testdata_path / "sat" / "ascat" / "netcdf" / "grid" / "TUW_WARP5_grid_info_2_1.nc"
static_layer_path = testdata_path / "sat" / "h_saf" / "static_layer"


#init the AscatSsmCdr reader with the paths
with warnings.catch_warnings():
    warnings.filterwarnings('ignore') # some warnings are expected and ignored
    
    ascat_reader = AscatGriddedNcTs(
        ascat_data_folder,
        "TUW_METOP_ASCAT_WARP55R22_{:04d}",
        grid_filename=ascat_grid_fname,
        static_layer_path=static_layer_path
    )

Create the ismn reader:

In [ ]:
#set path to ISMN data
path_to_ismn_data = testdata_path / "ismn" / "multinetwork" / "header_values"

#Initialize reader
ISMN_reader = ismn.ISMN_Interface(path_to_ismn_data)
list(ISMN_reader.stations_that_measure('soil moisture'))

We will compare only the first station to ASCAT here. For this station, we will compare ASCAT to the available measured time series from depths above 10 cm (which is only one in this case).

In [ ]:
station = next(ISMN_reader.stations_that_measure('soil moisture'))
list(station.data_for_variable('soil moisture', min_depth=0, max_depth=0.1))

We will first temporally collocate the ISMN time series to ASCAT. Then we will perform a CDF matching so that biases between the two will be removed.

In [ ]:
label_ascat='sm'
label_insitu='insitu_sm'



ISMN_time_series = next(station.data_for_variable('soil moisture', min_depth=0, max_depth=0.1))
ascat_time_series = ascat_reader.read(ISMN_time_series.longitude,
                                      ISMN_time_series.latitude,
                                      mask_ssf=True,
                                      mask_frozen_prob = 5,
                                      mask_snow_prob = 5).tz_localize("UTC")
ascat_time_series

In [ ]:
ISMN_time_series.data

We will rename the soil moisture column from ISMN so it's easier to differentiate them in plots. Also, drop all the NaNs here, they might lead to problems further on.

In [ ]:
ismn_sm = ISMN_time_series.data[["soil moisture"]].dropna()
ismn_sm.rename(columns={'soil moisture':label_insitu}, inplace=True)
ascat_sm = ascat_time_series[["sm"]].dropna()
ascat_sm

In [ ]:
ismn_sm

Now we need to temporally collocate the two time series. We do this using the nearest neighbour within +- 1 hour.

In [ ]:
matched_ismn = temp_match.temporal_collocation(ascat_sm, ismn_sm, pd.Timedelta(1, "H"))
matched_data = pd.concat((ascat_sm, matched_ismn), axis=1).dropna()
matched_data

In [ ]:
fig1, ax1 = plt.subplots()
matched_data.plot(figsize=(15,5),secondary_y=[label_ascat],
                  title='temporally merged data', ax=ax1);

There is still a bias between the time series, especially at the start. We can remove it by scaling, here we use CDF matching as a nonlinear scaling method.

In [ ]:
# this takes the matched_data DataFrame and scales all columns to the
# column with the given reference_index, in this case in situ
scaled_data = scaling.scale(matched_data, method='cdf_beta_match',
                            reference_index=1)

# now the scaled ascat data and insitu_sm are in the same space
fig2, ax2 = plt.subplots()
scaled_data.plot(figsize=(15,5), title='scaled data', ax=ax2);

To see the correlation, we can create a scatterplot.

In [ ]:
fig3, ax3 = plt.subplots()
ax3.scatter(scaled_data[label_ascat].values, scaled_data[label_insitu].values)
ax3.set_xlabel(label_ascat)
ax3.set_ylabel(label_insitu);

We can also calculate the correlation and other interesting metrics:

In [ ]:
# calculate correlation coefficients, RMSD, bias, Nash Sutcliffe
x, y = scaled_data[label_ascat].values, scaled_data[label_insitu].values

from scipy import stats
print("Pearson's R    = {:.2f}, p = {:.2e}".format(*stats.pearsonr(x, y)))
print("Spearman's rho = {:.2f}, p = {:.2e}".format(*stats.spearmanr(x, y)))
print("Kendall's tau  = {:.2f}, p = {:.2e}".format(*stats.kendalltau(x, y)))
print()
print("RMSD = {:.2f}".format(metrics.rmsd(x, y)))
print("Bias = {:.2f}".format(metrics.bias(x, y)))
print("Nash Sutcliffe = {:.2f}".format(metrics.nash_sutcliffe(x, y)))

The correlations are all significant, although there are only in the medium range. The bias is zero, because we scaled the data and thereby removed the bias.